In [ ]:
!pip install opencv-python torch torchvision torchaudio
!pip install ultralytics
!pip install moviepy
!pip install pydub
!apt-get install -y ffmpeg
!pip install gTTS
!pip install pyngrok
!pip install Streamlit
!sudo apt-get update -qq
!sudo apt-get install -y espeak-ng


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
import cv2
import os
import torch
from ultralytics import YOLO
from transformers import AutoTokenizer, AutoModelForCausalLM
from moviepy.editor import VideoFileClip, AudioFileClip
from pydub import AudioSegment
from gtts import gTTS
from huggingface_hub import login

# ✅ Mount Google Drive for model + cache
from google.colab import drive
drive.mount('/content/drive')

# Log in to Hugging Face Hub (Optional, for private models or higher rate limits)
try:
    login(new_session=False)
except Exception as e:
    print(f"Hugging Face login failed: {e}. Proceeding without login.")

# ------------------ Load Models Once ------------------
print("⏳ Loading YOLO + Gemma models...")

# YOLO models
model_coco   = YOLO("yolov8n.pt")
model_zebra  = YOLO("/content/drive/MyDrive/best (4).pt")
model_light  = YOLO("/content/drive/MyDrive/traffic_light.pt")

# ✅ Hugging Face Gemma with Google Drive caching
gemma_model_id = "google/gemma-2b-it"
save_path = "/content/drive/MyDrive/hf_models/gemma-2b-it"

# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


if os.path.exists(save_path):
    try:
        print("⚡ Loading Gemma model from Google Drive cache...")
        tokenizer = AutoTokenizer.from_pretrained(save_path, token=True)
        gemma_model = AutoModelForCausalLM.from_pretrained(
            save_path,
            torch_dtype=torch.float16,
        )
        gemma_model.to(device)
        print("✅ Gemma model loaded from cache!")
    except Exception as e:
        print(f"⚠️ Cache error: {e}, downloading instead...")
        tokenizer = AutoTokenizer.from_pretrained(gemma_model_id, token=True)
        gemma_model = AutoModelForCausalLM.from_pretrained(
            gemma_model_id,
            torch_dtype=torch.float16,
        )
        gemma_model.to(device)
else:
    print("📥 First time: downloading Gemma model...")
    tokenizer = AutoTokenizer.from_pretrained(gemma_model_id, token=True)
    gemma_model = AutoModelForCausalLM.from_pretrained(
        gemma_model_id,
        torch_dtype=torch.float16,
    )
    gemma_model.to(device)
    tokenizer.save_pretrained(save_path)
    gemma_model.save_pretrained(save_path)
    print("✅ Gemma model cached in Google Drive:", save_path)

print("✅ All models loaded successfully!")


# ------------------ Main Function ------------------
def run_narrated_detection(
    input_mode="video",
    input_path=None,
    output_path=None,
    cooldown=3
):
    if input_mode == "video" and input_path is not None:
        base, ext = os.path.splitext(os.path.basename(input_path))
        output_path = f"/content/{base}_final_output.mp4"
    elif output_path is None:
        output_path = "/content/final_output1.mp4"

    temp_video_path = "/content/temp_video.avi"
    audio_folder = "/content/temp_audio"
    os.makedirs(audio_folder, exist_ok=True)

    window_width, window_height = 1280, 720
    audio_counter = 0
    video_writer = None
    last_narration_time = -5
    fps = 20

    # ------------------ Helpers ------------------
    def save_tts_audio(text):
        nonlocal audio_counter
        audio_path = os.path.join(audio_folder, f"tts_{audio_counter}.mp3")
        tts = gTTS(text=text, lang="en")
        tts.save(audio_path)
        audio_counter += 1
        return audio_path

    def merge_audios(folder, output_path="/content/final_audio.wav"):
        files = sorted(os.listdir(folder), key=lambda x: int(x.split("_")[1].split(".")[0]))
        combined = AudioSegment.empty()
        for f in files:
            audio = AudioSegment.from_file(os.path.join(folder, f), format="mp3")
            combined += audio + AudioSegment.silent(duration=300)
        combined.export(output_path, format="wav")
        return output_path

    def format_counts(class_counts):
        parts = []
        for lbl, cnt in class_counts.items():
            if cnt == 1:
                parts.append(f"one {lbl}")
            else:
                parts.append(f"{cnt} {lbl}s")
        return ", ".join(parts)

    # ------------------ Frame Processing ------------------
    def process_frame(frame, current_time):
        nonlocal video_writer, last_narration_time, fps

        frame = cv2.resize(frame, (window_width, window_height))
        results_coco = model_coco(frame)
        results_zebra = model_zebra(frame)
        results_light = model_light(frame)

        annotated = frame.copy()
        class_counts, zebra_count, light_color = {}, 0, None

        # Traffic light
        for box in results_light[0].boxes:
            cls = int(box.cls[0])
            label = model_light.names[cls]
            conf = float(box.conf[0])
            if conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(annotated, (x1, y1), (x2, y2), (255, 255, 0), 2)
                cv2.putText(annotated, f"Light: {label}", (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
                light_color = label
                break

        # Zebra crossings
        for box in results_zebra[0].boxes:
            if float(box.conf[0]) > 0.3:
                zebra_count += 1

        # Vehicles + people
        for box in results_coco[0].boxes:
            cls = int(box.cls[0])
            label = model_coco.names[cls]
            if float(box.conf[0]) > 0.6 and label in ["car", "bus", "truck", "motorcycle", "person"]:
                class_counts[label] = class_counts.get(label, 0) + 1

        # Scene description
        desc = []
        if light_color:
            desc.append(f"traffic light is {light_color.lower()}")
        if zebra_count > 0:
            desc.append(f"{zebra_count} zebra crossing{'s' if zebra_count > 1 else ''} ahead")
        if class_counts:
            desc.append(f"{format_counts(class_counts)} ahead")

        # Narration
        if desc and (current_time - last_narration_time >= cooldown):
            user_prompt = (
    "You are a human walking guide helping a blind pedestrian. "
    f"Environment: {', '.join(desc)}. "
    "Give ONLY one short, natural instruction in plain English. "
    "Speak as if you are standing next to them. "
    "Examples: 'Wait, red light ahead', 'Cross now at the zebra crossing', 'Walk forward carefully'. "
    "🚫 Do NOT mention AI, detection, models, systems, or technology. "
    "🚫 Do NOT describe the task, just give the instruction."
)
            messages = [{"role": "user", "content": user_prompt}]
            chat_prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                chat_template=tokenizer.chat_template
            )

            inputs = tokenizer(chat_prompt, return_tensors="pt").to(gemma_model.device)
            outputs = gemma_model.generate(**inputs, max_new_tokens=40, temperature=0.7, top_p=0.9)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Strip prompt echo
            #if user_prompt in response:
             #   response = response.split(user_prompt)[-1].strip()

            # Remove unwanted junk
            #bad_phrases = ["model is detecting", "detected", "system sees", "AI","model"]
            #for phrase in bad_phrases:
             #   response = response.replace(phrase, "").strip()

            # Strip prompt echo
            if user_prompt in response:
                response = response.split(user_prompt)[-1].strip()

            # Remove junk / tech words
            ban_words = [
                    "model", "AI", "system", "detection", "detected", "algorithm",
                    "neural", "processing", "technology", "predicting"
                            ]
            for word in ban_words:
                response = response.replace(word, "").strip()

            # Extra cleanup
            response = response.replace("  ", " ").replace("..", ".").strip()


            # Fallback if Gemma fails
            if not response or len(response.split()) < 2:
                if light_color == "red":
                    response = "Stop, red light ahead"
                elif light_color == "green":
                    response = "Cross now, green light"
                elif zebra_count > 0:
                    response = "Cross at the zebra crossing"
                else:
                    response = "Walk carefully ahead"

            print(f"[{current_time:.1f}s] 🤖 Narration:", response)
            save_tts_audio(response)
            last_narration_time = current_time

        # Write video
        if video_writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"XVID")
            h, w, _ = annotated.shape
            video_writer = cv2.VideoWriter(temp_video_path, fourcc, fps, (w, h))
        video_writer.write(annotated)
        return annotated

    # ------------------ Modes ------------------
    if input_mode == "image":
        img = cv2.imread(input_path)
        if img is not None:
            process_frame(img, 0)
        else:
            print("⚠️ Could not read image.")

    elif input_mode == "video":
        cap = cv2.VideoCapture(input_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps == 0: fps = 20
        frame_idx = 0
        while True:
            ret, frame = cap.read()
            if not ret: break
            current_time = frame_idx / fps
            process_frame(frame, current_time)
            frame_idx += 1
        cap.release()

    # Finalize video
    if video_writer is not None:
        video_writer.release()

    final_audio_path = merge_audios(audio_folder, "/content/temp_audio.wav")
    video_clip = VideoFileClip(temp_video_path)
    audio_clip = AudioFileClip(final_audio_path)
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    print("✅ Final narrated video saved:", output_path)
    return output_path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



⏳ Loading YOLO + Gemma models...
Using device: cuda
⚡ Loading Gemma model from Google Drive cache...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

⚠️ Cache error: No such file or directory: /content/drive/MyDrive/hf_models/gemma-2b-it/model-00001-of-00002.safetensors, downloading instead...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ All models loaded successfully!


In [ ]:
run_narrated_detection(input_mode="image", input_path="/content/sam_IMG_2466_JPG.rf.216a17fbbd6d2196e58eedba87236945.jpg")


0: 384x640 6 cars, 1 traffic light, 1074.5ms
Speed: 21.7ms preprocess, 1074.5ms inference, 55.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Reds, 696.1ms
Speed: 6.4ms preprocess, 696.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Reds, 1355.1ms
Speed: 5.1ms preprocess, 1355.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[0.0s] 🤖 Narration: Stop, wait for the light to turn green, and cross the zebra crossing when it does.
Moviepy - Building video /content/final_output1.mp4.
MoviePy - Writing audio in final_output1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/final_output1.mp4



Moviepy - Done !
Moviepy - video ready /content/final_output1.mp4
✅ Final narrated video saved: /content/final_output1.mp4


'/content/final_output1.mp4'

In [ ]:
# For video
run_narrated_detection(input_mode="video", input_path="/content/drive/MyDrive/traffic_video.mp4")


0: 384x640 4 persons, 8 cars, 2 trucks, 9 traffic lights, 80.9ms
Speed: 7.1ms preprocess, 80.9ms inference, 295.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Red, 17.1ms
Speed: 3.3ms preprocess, 17.1ms inference, 155.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Red, 11.4ms
Speed: 1.6ms preprocess, 11.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[0.0s] 🤖 Narration: Wait, red light ahead. Cross now at the zebra crossing.

0: 384x640 4 persons, 8 cars, 2 trucks, 9 traffic lights, 8.0ms
Speed: 3.7ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Red, 9.5ms
Speed: 2.2ms preprocess, 9.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Red, 7.5ms
Speed: 2.9ms preprocess, 7.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 8 cars, 3 trucks, 7 traffic lights, 7.7ms
Speed: 3.2ms preprocess, 7.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Red, 7.4ms
Speed: 2.3ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Red, 6.7ms
Speed: 2.1ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 8 cars, 3 trucks, 8 traffic lights, 7.7ms
Speed: 3.2ms preprocess, 7.7ms inference, 1.6ms postp

MoviePy - Done.
Moviepy - Writing video /content/traffic_video_final_output.mp4



Moviepy - Done !
Moviepy - video ready /content/traffic_video_final_output.mp4
✅ Final narrated video saved: /content/traffic_video_final_output.mp4


'/content/traffic_video_final_output.mp4'

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
%%writefile run_detection.py
import cv2
import os
import torch
from ultralytics import YOLO
from transformers import AutoTokenizer, AutoModelForCausalLM
from moviepy.editor import VideoFileClip, AudioFileClip
from pydub import AudioSegment
from gtts import gTTS
from huggingface_hub import login



# Log in to Hugging Face Hub (Optional, for private models or higher rate limits)
try:
    login(new_session=False)
except Exception as e:
    print(f"Hugging Face login failed: {e}. Proceeding without login.")

# ------------------ Load Models Once ------------------
print("⏳ Loading YOLO + Gemma models...")

# YOLO models
model_coco   = YOLO("yolov8n.pt")
model_zebra  = YOLO("/content/drive/MyDrive/best (4).pt")
model_light  = YOLO("/content/drive/MyDrive/traffic_light.pt")

# ✅ Hugging Face Gemma with Google Drive caching
gemma_model_id = "google/gemma-2b-it"
save_path = "/content/drive/MyDrive/hf_models/gemma-2b-it"

# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


if os.path.exists(save_path):
    try:
        print("⚡ Loading Gemma model from Google Drive cache...")
        tokenizer = AutoTokenizer.from_pretrained(save_path, token=True)
        gemma_model = AutoModelForCausalLM.from_pretrained(
            save_path,
            torch_dtype=torch.float16,
        )
        gemma_model.to(device)
        print("✅ Gemma model loaded from cache!")
    except Exception as e:
        print(f"⚠️ Cache error: {e}, downloading instead...")
        tokenizer = AutoTokenizer.from_pretrained(gemma_model_id, token=True)
        gemma_model = AutoModelForCausalLM.from_pretrained(
            gemma_model_id,
            torch_dtype=torch.float16,
        )
        gemma_model.to(device)
else:
    print("📥 First time: downloading Gemma model...")
    tokenizer = AutoTokenizer.from_pretrained(gemma_model_id, token=True)
    gemma_model = AutoModelForCausalLM.from_pretrained(
        gemma_model_id,
        torch_dtype=torch.float16,
    )
    gemma_model.to(device)
    tokenizer.save_pretrained(save_path)
    gemma_model.save_pretrained(save_path)
    print("✅ Gemma model cached in Google Drive:", save_path)

print("✅ All models loaded successfully!")


# ------------------ Main Function ------------------
def run_narrated_detection(
    input_mode="video",
    input_path=None,
    output_path=None,
    cooldown=3
):
    if input_mode == "video" and input_path is not None:
        base, ext = os.path.splitext(os.path.basename(input_path))
        output_path = f"/content/{base}_final_output.mp4"
    elif output_path is None:
        output_path = "/content/final_output1.mp4"

    temp_video_path = "/content/temp_video.avi"
    audio_folder = "/content/temp_audio"
    os.makedirs(audio_folder, exist_ok=True)

    window_width, window_height = 1280, 720
    audio_counter = 0
    video_writer = None
    last_narration_time = -5
    fps = 20

    # ------------------ Helpers ------------------
    def save_tts_audio(text):
        nonlocal audio_counter
        audio_path = os.path.join(audio_folder, f"tts_{audio_counter}.mp3")
        tts = gTTS(text=text, lang="en")
        tts.save(audio_path)
        audio_counter += 1
        return audio_path

    def merge_audios(folder, output_path="/content/final_audio.wav"):
        files = sorted(os.listdir(folder), key=lambda x: int(x.split("_")[1].split(".")[0]))
        combined = AudioSegment.empty()
        for f in files:
            audio = AudioSegment.from_file(os.path.join(folder, f), format="mp3")
            combined += audio + AudioSegment.silent(duration=300)
        combined.export(output_path, format="wav")
        return output_path

    def format_counts(class_counts):
        parts = []
        for lbl, cnt in class_counts.items():
            if cnt == 1:
                parts.append(f"one {lbl}")
            else:
                parts.append(f"{cnt} {lbl}s")
        return ", ".join(parts)

    # ------------------ Frame Processing ------------------
    def process_frame(frame, current_time):
        nonlocal video_writer, last_narration_time, fps

        frame = cv2.resize(frame, (window_width, window_height))
        results_coco = model_coco(frame)
        results_zebra = model_zebra(frame)
        results_light = model_light(frame)

        annotated = frame.copy()
        class_counts, zebra_count, light_color = {}, 0, None

        # Traffic light
        for box in results_light[0].boxes:
            cls = int(box.cls[0])
            label = model_light.names[cls]
            conf = float(box.conf[0])
            if conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(annotated, (x1, y1), (x2, y2), (255, 255, 0), 2)
                cv2.putText(annotated, f"Light: {label}", (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
                light_color = label
                break

        # Zebra crossings
        for box in results_zebra[0].boxes:
            if float(box.conf[0]) > 0.3:
                zebra_count += 1

        # Vehicles + people
        for box in results_coco[0].boxes:
            cls = int(box.cls[0])
            label = model_coco.names[cls]
            if float(box.conf[0]) > 0.6 and label in ["car", "bus", "truck", "motorcycle", "person"]:
                class_counts[label] = class_counts.get(label, 0) + 1

        # Scene description
        desc = []
        if light_color:
            desc.append(f"traffic light is {light_color.lower()}")
        if zebra_count > 0:
            desc.append(f"{zebra_count} zebra crossing{'s' if zebra_count > 1 else ''} ahead")
        if class_counts:
            desc.append(f"{format_counts(class_counts)} ahead")

        # Narration
        if desc and (current_time - last_narration_time >= cooldown):
            user_prompt = (
    "You are a human walking guide helping a blind pedestrian. "
    f"Environment: {', '.join(desc)}. "
    "Give ONLY one short, natural instruction in plain English. "
    "Speak as if you are standing next to them. "
    "Examples: 'Wait, red light ahead', 'Cross now at the zebra crossing', 'Walk forward carefully'. "
    "🚫 Do NOT mention AI, detection, models, systems, or technology. "
    "🚫 Do NOT describe the task, just give the instruction."
)
            messages = [{"role": "user", "content": user_prompt}]
            chat_prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                chat_template=tokenizer.chat_template
            )

            inputs = tokenizer(chat_prompt, return_tensors="pt").to(gemma_model.device)
            outputs = gemma_model.generate(**inputs, max_new_tokens=40, temperature=0.7, top_p=0.9)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Strip prompt echo
            #if user_prompt in response:
             #   response = response.split(user_prompt)[-1].strip()

            # Remove unwanted junk
            #bad_phrases = ["model is detecting", "detected", "system sees", "AI","model"]
            #for phrase in bad_phrases:
             #   response = response.replace(phrase, "").strip()

            # Strip prompt echo
            if user_prompt in response:
                response = response.split(user_prompt)[-1].strip()

            # Remove junk / tech words
            ban_words = [
                    "model", "AI", "system", "detection", "detected", "algorithm",
                    "neural", "processing", "technology", "predicting"
                            ]
            for word in ban_words:
                response = response.replace(word, "").strip()

            # Extra cleanup
            response = response.replace("  ", " ").replace("..", ".").strip()


            # Fallback if Gemma fails
            if not response or len(response.split()) < 2:
                if light_color == "red":
                    response = "Stop, red light ahead"
                elif light_color == "green":
                    response = "Cross now, green light"
                elif zebra_count > 0:
                    response = "Cross at the zebra crossing"
                else:
                    response = "Walk carefully ahead"

            print(f"[{current_time:.1f}s] 🤖 Narration:", response)
            save_tts_audio(response)
            last_narration_time = current_time

        # Write video
        if video_writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"XVID")
            h, w, _ = annotated.shape
            video_writer = cv2.VideoWriter(temp_video_path, fourcc, fps, (w, h))
        video_writer.write(annotated)
        return annotated

    # ------------------ Modes ------------------
    if input_mode == "image":
        img = cv2.imread(input_path)
        if img is not None:
            process_frame(img, 0)
        else:
            print("⚠️ Could not read image.")

    elif input_mode == "video":
        cap = cv2.VideoCapture(input_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps == 0: fps = 20
        frame_idx = 0
        while True:
            ret, frame = cap.read()
            if not ret: break
            current_time = frame_idx / fps
            process_frame(frame, current_time)
            frame_idx += 1
        cap.release()

    # Finalize video
    if video_writer is not None:
        video_writer.release()

    final_audio_path = merge_audios(audio_folder, "/content/temp_audio.wav")
    video_clip = VideoFileClip(temp_video_path)
    audio_clip = AudioFileClip(final_audio_path)
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    print("✅ Final narrated video saved:", output_path)
    return output_path



Overwriting run_detection.py


In [ ]:
%%writefile utils.py
import streamlit as st
import base64

def add_bg_from_local(image_file):
    """Embed background image in base64 to avoid fetch errors"""
    with open(image_file, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()

    css_code = f"""
    <style>
    .stApp {{
        background: url("data:image/png;base64,{encoded}");
        background-size: cover;
        background-position: center;
        background-attachment: fixed;
    }}
    </style>
    """
    st.markdown(css_code, unsafe_allow_html=True)

def set_theme():
    # Initialize session state if not set
    if "theme_choice" not in st.session_state:
        st.session_state.theme_choice = "⚡ Default"

    theme_choice = st.sidebar.radio(
        "🎨 Choose Theme",
        ["🌞 Light", "🌙 Dark", "⚡ Default"],
        index=["🌞 Light", "🌙 Dark", "⚡ Default"].index(st.session_state.theme_choice)
    )

    # Save selection to session state
    st.session_state.theme_choice = theme_choice

    # Apply theme CSS
    if theme_choice == "🌞 Light":
        st.markdown("""
            <style>
                body, .stApp { background-color: rgba(255,255,255,0.2) !important; color: #000 !important; }
                .card { background: rgba(255,255,255,0.85) !important; color: #000 !important; }
                h1, h2, h3, p { color: #2c3e50 !important; }
            </style>
        """, unsafe_allow_html=True)
    elif theme_choice == "🌙 Dark":
        st.markdown("""
            <style>
                body, .stApp { background-color: rgba(0,0,0,0.2) !important; color: #fff !important; }
                .card { background: rgba(0,0,0,0.85) !important; color: #f5f5f5 !important; }
                h1, h2, h3, p { color: #e0e0e0 !important; }
            </style>
        """, unsafe_allow_html=True)
    else:  # Default
        st.markdown("""
            <style>
                body, .stApp { background-color: transparent !important; }
                .card { background: rgba(0,0,0,0.75); }
            </style>
        """, unsafe_allow_html=True)



Writing utils.py


In [ ]:
%%writefile Home.py
import streamlit as st
from utils import add_bg_from_local, set_theme

# ---------------- Background Setup ----------------
add_bg_from_local("/content/drive/MyDrive/robin-pierre-dPgPoiUIiXk-unsplash.jpg")

st.set_page_config(page_title="Assistive Mobility Tool", page_icon="🤖", layout="wide")

# Apply theme
set_theme()

# ---------------- Global Styles ----------------
st.markdown("""
    <style>
        /* ===== Variables ===== */
        :root {
            --card-padding: 25px;
            --card-margin: 20px 0;
            --card-radius: 15px;
            --card-shadow: 4px 4px 12px rgba(0,0,0,0.5);
            --title-bg: rgba(0,0,0,0.85);
            --section-bg: rgba(15,15,15,0.7);
            --nav-bg: rgba(34,34,34,0.6);
        }

        /* ===== Title Glow ===== */
        .glow-text {
            font-size: 52px;
            font-weight: bold;
            background: linear-gradient(90deg, #00c6ff, #0072ff);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            animation: glow 2s ease-in-out infinite alternate;
        }
        @keyframes glow {
            from { text-shadow: 0 0 5px #00c6ff; }
            to { text-shadow: 0 0 12px #0072ff; }
        }

        /* ===== Fade-in Animation ===== */
        .fade-in {
            animation: fadeIn 1.5s ease-in;
        }
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }

        /* ===== Card Styling ===== */
        .card {
            padding: var(--card-padding);
            margin: var(--card-margin);
            border-radius: var(--card-radius);
            box-shadow: var(--card-shadow);
            color: #fff;
        }
        .title-card { text-align: center; }
        .section-card { font-size: 18px; line-height: 1.7; }
    </style>
""", unsafe_allow_html=True)

# ---------------- Title Card ----------------
st.markdown("""
    <div class="card title-card fade-in">
        <h1 class="glow-text">👁️‍🗨️ Assistive Mobility Tool</h1>
        <p style="font-size:22px;">AI Narrated Detection for the Visually Impaired</p>
    </div>
""", unsafe_allow_html=True)

# ---------------- How it Works Section ----------------
st.markdown("""
    <div class="card section-card fade-in">
        <h2>⚙️ How It Works</h2>
        <div style="margin-top: 15px;">
            <p>✅ <b>Step 1:</b> Upload an <b>image, video, or webcam capture</b> 📸</p>
            <p>✅ <b>Step 2:</b> System runs <b>YOLO for object detection</b> 🎯</p>
            <p>✅ <b>Step 3:</b> Objects are <b>narrated via AI voice</b> 🔊</p>
            <p>✅ <b>Step 4:</b> Get <b>preview & download</b> 📂</p>
        </div>
    </div>
""", unsafe_allow_html=True)

# ---------------- Navigation Buttons ----------------
with st.container():
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        if st.button("🚀 Run Detection"):
            st.switch_page("pages/1_Detection.py")
    with col2:
        if st.button("🌐 Technologies Use"):
            st.switch_page("pages/2_About.py")
    with col3:
        if st.button("📊 Future Scope"):
            st.switch_page("pages/2_About.py")
    with col4:
        if st.button("👩🏻‍💻 Developer Details"):
            st.switch_page("pages/2_About.py")


Writing Home.py


In [ ]:
!mkdir -p pages


In [ ]:
%%writefile pages/1AI_Narrator.py
import streamlit as st
from utils import add_bg_from_local, set_theme
from run_detection import run_narrated_detection
from moviepy.editor import VideoFileClip
import os, mimetypes

# Background
add_bg_from_local("/content/drive/MyDrive/pietro-jeng-n6B49lTx7NM-unsplash.jpg")

st.set_page_config(page_title="Run Detection", layout="wide")

# Apply theme
set_theme()

# ---------------- Global Styles ----------------
st.markdown("""
    <style>
        .glow-text {
            font-size: 52px;
            font-weight: bold;
            background: linear-gradient(90deg, #00c6ff, #0072ff);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            animation: glow 3s ease-in-out infinite alternate;
        }
        @keyframes glow {
            from { text-shadow: 0 0 3px #00c6ff; }
            to { text-shadow: 0 0 10px #0072ff; }
        }

        .fade-in {
            animation: fadeIn 1.5s ease-in;
        }
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .card {
            padding: 25px;
            margin: 20px 0;
            border-radius: 15px;
            box-shadow: 4px 4px 12px rgba(0,0,0,0.5);
            background: rgba(0,0,0,0.75);
            color: #fff;
        }
    </style>
""", unsafe_allow_html=True)

# ---------------- Title ----------------
st.markdown(
    """
    <div class="card fade-in" style="text-align:center;">
        <h1 class="glow-text">🚀 Run Detection</h1>
        <p style="font-size:20px;">Upload input and let AI assist with narrated detection</p>
    </div>
    """,
    unsafe_allow_html=True
)

# ---------------- Input Options ----------------
mode = st.sidebar.radio("📂 Select Input Mode", ["🎥 Video", "🖼️ Image", "📸 Webcam"])
cooldown = st.sidebar.slider("⏳ Narration Cooldown (seconds)", 1, 10, 3)

input_path = None

if "Video" in mode:
    uploaded_file = st.file_uploader("🎥 Upload a video", type=["mp4", "avi", "mov"])
    if uploaded_file:
        input_path = uploaded_file.name
        with open(input_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
        st.success("✅ Video uploaded successfully!")

elif "Image" in mode:
    uploaded_file = st.file_uploader("🖼️ Upload an image", type=["jpg", "jpeg", "png"])
    if uploaded_file:
        input_path = uploaded_file.name
        with open(input_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
        st.success("✅ Image uploaded successfully!")

elif "Webcam" in mode:
    st.subheader("📸 Webcam Options")

    # ---- Capture Image ----
    captured_image = st.camera_input("📸 Capture from webcam (Image)")
    if captured_image:
        input_path = "webcam_capture.jpg"
        with open(input_path, "wb") as f:
            f.write(captured_image.getbuffer())
        st.success("✅ Webcam image captured!")

    # ---- Placeholder for Video ----
    st.markdown(
        """
        <div style="margin-top:15px; padding:15px; border:2px dashed #888; border-radius:10px; text-align:center; color:#bbb;">
            🎥 Webcam video capture coming soon...
        </div>
        """,
        unsafe_allow_html=True
    )

# -------- Run Detection --------
if st.button("🚀 Run Detection"):
    if not input_path:
        st.error(f"⚠️ Please provide a {mode} input first.")
    else:
        with st.spinner("🔎 Processing input... Please wait ⏳"):
            output_path = run_narrated_detection(
                input_mode=mode.lower().replace("📸 ","").replace("🎥 ","").replace("🖼️ ","").strip() if "Webcam" not in mode else "image",
                input_path=input_path,
                output_path=None,
                cooldown=cooldown
            )

        if output_path and os.path.exists(output_path):
            st.success("✅ Narrated output generated!")

            # -------- Extract Audio --------
            audio_path = None
            mime_type, _ = mimetypes.guess_type(output_path)

            if mime_type:
                if mime_type.startswith("video"):
                    try:
                        clip = VideoFileClip(output_path)
                        audio_path = os.path.splitext(output_path)[0] + ".mp3"
                        clip.audio.write_audiofile(audio_path)
                        clip.close()
                        st.info("🎵 Audio narration extracted successfully!")
                    except Exception as e:
                        st.warning(f"⚠️ Could not extract audio: {e}")

                elif mime_type.startswith("audio"):
                    audio_path = output_path  # already audio

            # -------- Tabs --------
            tab1, tab2 = st.tabs(["📺 Preview", "⬇️ Download"])

            with tab1:
                if mime_type:
                    if mime_type.startswith("image"):
                        st.image(output_path, use_column_width=True)
                        if audio_path and os.path.exists(audio_path):
                            st.audio(audio_path)
                    elif mime_type.startswith("video"):
                        st.video(output_path)
                        if audio_path and os.path.exists(audio_path):
                            st.audio(audio_path)
                    elif mime_type.startswith("audio"):
                        st.audio(output_path)
                    else:
                        st.write("📂 File generated:", output_path)

            with tab2:
                with open(output_path, "rb") as f:
                    st.download_button(
                        label="⬇️ Download Final Output",
                        data=f,
                        file_name=os.path.basename(output_path),
                        mime=mimetypes.guess_type(output_path)[0] or "application/octet-stream",
                        use_container_width=True
                    )
                if audio_path and os.path.exists(audio_path) and audio_path != output_path:
                    with open(audio_path, "rb") as f:
                        st.download_button(
                            label="⬇️ Download Narration (Audio Only)",
                            data=f,
                            file_name=os.path.basename(audio_path),
                            mime="audio/mpeg",
                            use_container_width=True
                        )
        else:
            st.error("❌ Failed to generate output. Please check logs.")


Writing pages/1AI_Narrator.py


In [ ]:
%%writefile pages/2_About.py
import streamlit as st
from utils import add_bg_from_local, set_theme

# ---------------- Background ----------------
add_bg_from_local("/content/drive/MyDrive/pexels-isaquepereira-394377.jpg")

st.set_page_config(page_title="About Project", layout="wide")

# Apply theme
set_theme()

# ---------------- Global Styles ----------------
st.markdown("""
<style>
    /* ===== Glow Title ===== */
    .glow-text {
        font-size: 48px;
        font-weight: bold;
        background: linear-gradient(90deg, #00c6ff, #0072ff);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        animation: glow 2s ease-in-out infinite alternate;
    }
    @keyframes glow {
        from { text-shadow: 0 0 5px #00c6ff; }
        to { text-shadow: 0 0 12px #0072ff; }
    }

    /* ===== Fade-in Animation ===== */
    .fade-in {
        animation: fadeIn 1.5s ease-in;
    }
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(20px); }
        to { opacity: 1; transform: translateY(0); }
    }

    /* ===== Transparent Card Styling ===== */
    .transparent-card {
        background-color: rgba(0,0,0,0.75);
        color: #fff;
        padding: 25px;
        border-radius: 15px;
        box-shadow: 4px 4px 10px rgba(0,0,0,0.5);
        margin: 20px 0;
        font-size: 18px;
    }
    .transparent-card h2 {
        color: #00c6ff; /* match Home.py accent */
    }
    .transparent-card ul li {
        margin-bottom: 8px;
    }
    .email-box {
        background: rgba(255,255,255,0.15);
        padding: 5px 10px;
        border-radius: 8px;
        display: inline-block;
        font-family: monospace;
    }
</style>
""", unsafe_allow_html=True)

# ---------------- Title ----------------
st.markdown("""
<div class="fade-in" style="text-align:center; font-size:48px; font-weight:bold; margin-bottom:20px;">
    ℹ️ <span class="glow-text">About This Project</span>
</div>
""", unsafe_allow_html=True)

# ---------------- Project Usefulness ----------------
st.markdown("""
<div class="transparent-card fade-in">
    <h2>📌 Project Usefulness</h2>
    <ul>
        <li>Detecting obstacles in their surroundings 👀</li>
        <li>Providing <b>real-time narration</b> 🔊</li>
        <li>Making mobility safer and more independent 🚶</li>
        <li>Enhancing <b>awareness of the environment</b> 🏞️</li>
        <li>Supporting <b>indoor and outdoor navigation</b> 🏠➡️🌳</li>
        <li>Reducing dependency on others for everyday mobility 🙌</li>
        <li>Offering <b>customizable narration speed and cooldown</b> ⏳</li>
        <li>Capturing input from multiple modes (image, video, webcam) 🎥🖼️📸</li>
        <li>Generating <b>audio descriptions</b> for better accessibility 🎧</li>
        <li>Enabling integration with wearable devices or mobile apps 📱</li>
        <li>Improving confidence, inclusivity, and quality of life ❤️</li>
    </ul>
</div>
""", unsafe_allow_html=True)

# ---------------- Technologies Used ----------------
st.markdown("""
<div class="transparent-card fade-in">
    <h2>💻 Technologies Used</h2>
    <ul>
        <li><b>Python</b> – Core programming language 🐍</li>
        <li><b>Streamlit</b> – Interactive web interface ⚡</li>
        <li><b>YOLOv5</b> – State-of-the-art object detection 🎯</li>
        <li><b>OpenCV</b> – Image & video processing 📷</li>
        <li><b>gTTS / pyttsx3</b> – AI-based speech narration 🔊</li>
        <li><b>MoviePy</b> – Extracting and handling audio/video 🎥🎵</li>
    </ul>
</div>
""", unsafe_allow_html=True)

# ---------------- YOLO Model Explained ----------------
st.markdown("""
<div class="transparent-card fade-in">
    <h2>🔍 YOLO Model (You Only Look Once)</h2>
    <p>
        YOLO is a <b>real-time object detection algorithm</b> that processes an entire image in a single pass.
        Instead of scanning parts of the image multiple times, YOLO divides the image into a grid and predicts:
    </p>
    <ul>
        <li>Bounding boxes around objects 📦</li>
        <li>Class labels (e.g., person, car, dog) 🏷️</li>
        <li>Confidence scores for detection ✅</li>
    </ul>
    <p>
        <b>Why YOLO?</b><br>
        ⚡ Extremely fast & accurate <br>
        🎯 Works on both images and videos <br>
        🔊 Perfect for real-time narration in assistive tools
    </p>
</div>
""", unsafe_allow_html=True)

# ---------------- Future Scope ----------------
st.markdown("""
<div class="transparent-card fade-in">
    <h2>🚀 Future Scope</h2>
    <ul>
        <li>GPS integration for real-time navigation with audio guidance 🗺️</li>
        <li>Voice commands for hands-free interaction 🎤</li>
        <li>AR/Smart glasses integration for seamless obstacle detection 🕶️</li>
        <li>Multilingual narration support 🌍</li>
        <li>Cloud-based processing to reduce device computation load ☁️</li>
        <li>Emergency assistance feature 🚨</li>
        <li>Machine learning improvements for complex environments 🤖</li>
        <li>Integration with public transport systems 🚍🚆</li>
    </ul>
</div>
""", unsafe_allow_html=True)

# ---------------- Developers ----------------
st.markdown("""
<div class="transparent-card fade-in">
    <h2>👨‍💻 Project Developer</h2>
    <ul>
        <li><b>Maharajmaran G</b> – Project Lead, ML Engineer & Data Scientist 💻</li>
    </ul>
    <p><b>Guided by:</b> [Mentor / Professor Name]</p>
    <p>
        <a href="https://github.com/maran103" style="color:#1DA1F2;">🌐 GitHub</a> |
        <a href="https://www.linkedin.com/in/maharajmaran-g-18684b257?" style="color:#1DA1F2;">🔗 LinkedIn</a>
    </p>
    <p>✉️ Email: <span class="email-box">maharajmaranoff@gmail.com</span></p>
</div>
""", unsafe_allow_html=True)


Overwriting pages/2_About.py


Overwriting pages/2_Detection.py


In [ ]:
!ngrok config add-authtoken 32jldxW1rJPSQDx8MSazouD9pxq_24dtsGNZ7xCecKFDeDZcB


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pkill -f ngrok


In [ ]:
!pip install streamlit cloudflared


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.7 MB/s eta 0:00:00
  Created wheel for cloudflared: filename=cloudflared-1.0.0.2-py3-none-any.whl size=2983 sha256=417ad39dd648b6f6fd66189f3ddc48429b6896ba397d674cdf6c6ce473ad1504
  Stored in directory: /root/.cache/pip/wheels/5b/ec/09/c3bcd3470be046ec77a9c0cb9d8bb6ceed49c831460878ab0a
Successfully built cloudflared


In [ ]:
from pyngrok import ngrok
import subprocess

# Kill any previous Streamlit process
!kill -9 $(lsof -t -i:8501) 2>/dev/null || echo "No existing Streamlit running"

# Start Streamlit in background
subprocess.Popen(["streamlit", "run", "Home.py", "--server.port", "8501"])

# Connect with ngrok (requires authtoken)
public_url = ngrok.connect(8501)
print("🌍 Streamlit app is live at:", public_url)


🌍 Streamlit app is live at: NgrokTunnel: "https://eac79ed9bcf5.ngrok-free.app" -> "http://localhost:8501"
